In [4]:
import sys
sys.path.append('../onitama/')
from trainer import RegularDataTrainer
from dl_players_v3 import CNNPlayer_v3
import numpy as np
from livelossplot import PlotLossesKeras
from tensorflow.keras.callbacks import EarlyStopping

ModuleNotFoundError: No module named 'dl_players'

In [2]:
def action_to_one_hot(action, shape=(5, 5, 52)):
    """
    Convertit une action [col, ligne, action_id] en one-hot tensor
    
    Args:
        action: [col, ligne, action_id] ou liste d'actions (batch)
        shape: (height, width, n_actions)
        
    Returns:
        one_hot: (5, 5, 52) ou (batch, 5, 5, 52)
    """
    if isinstance(action, list) and len(action) == 3:
        # Une seule action
        col, ligne, action_id = action
        one_hot = np.zeros(shape, dtype=np.float32)
        one_hot[col, ligne, action_id] = 1.0
        return one_hot
    else:
        # Batch d'actions
        batch_size = len(action)
        one_hot_batch = np.zeros((batch_size, *shape), dtype=np.float32)
        
        for i, act in enumerate(action):
            col, ligne, action_id = act
            one_hot_batch[i, col, ligne, action_id] = 1.0
        
        return one_hot_batch

def flat_index_to_action(flat_index):
    """
    Convertit index flat [0, 1299] en [col, ligne, move_id]
    """
    col = flat_index // (5 * 52)
    ligne = (flat_index // 52) % 5
    move_id = flat_index % 52
    return col, ligne, move_id


def decode_flat_policy(flat_policy):
    """
    Décode un vecteur aplati (1300,) en [col, ligne, move_id]
    
    Args:
        flat_policy: array de shape (1300,) - one-hot ou probabilités
        
    Returns:
        action: [col, ligne, move_id]
    """
    # 1. Trouver l'index du maximum (ou du 1.0 si one-hot)
    best_index = np.argmax(flat_policy)
    
    # 2. Décoder l'index
    col = best_index // (5 * 52)
    ligne = (best_index // 52) % 5
    move_id = best_index % 52
    
    return [int(col), int(ligne), int(move_id)]

def old_one_hot_to_action(action, verbose=False):
    """
    Convertit un one-hot tensor en action [col, ligne, move_id]
    
    Args:
        action: (5, 5, 52) ou (batch, 5, 5, 52)
        verbose: afficher les détails
        
    Returns:
        (col, ligne, move_id) ou liste de (col, ligne, move_id)
    """
    action = np.array(action)
    
    # Déterminer si c'est un batch ou une action seule
    if action.ndim == 3:
        # Une seule action (5, 5, 52)
        flat_logits = action.reshape(-1)
        
        exp_logits = np.exp(flat_logits - np.max(flat_logits))
        probabilities = exp_logits / exp_logits.sum()
        
        best_flat = np.argmax(probabilities)
        best_col, best_ligne, best_move_id = flat_index_to_action(best_flat)
        
        if verbose:
            print(f"Probabilités : {probabilities.shape}, sum={probabilities.sum():.3f}")
            print(f"Meilleure action : [{best_col}, {best_ligne}, {best_move_id}]")
            print(f"Probabilité : {probabilities[best_flat]:.4f}")
        
        return (best_col, best_ligne, best_move_id)
    
    elif action.ndim == 4:
        # Batch d'actions (batch, 5, 5, 52)
        batch_size = action.shape[0]
        results = []
        
        for i in range(batch_size):
            flat_logits = action[i].reshape(-1)
            
            exp_logits = np.exp(flat_logits - np.max(flat_logits))
            probabilities = exp_logits / exp_logits.sum()
            
            best_flat = np.argmax(probabilities)
            best_col, best_ligne, best_move_id = flat_index_to_action(best_flat)
            
            if verbose:
                print(f"[{i}] Probabilités : {probabilities.shape}, sum={probabilities.sum():.3f}")
                print(f"[{i}] Meilleure action : [{best_col}, {best_ligne}, {best_move_id}]")
                print(f"[{i}] Probabilité : {probabilities[best_flat]:.4f}")
            
            results.append((best_col, best_ligne, best_move_id))
        
        return results
    
    else:
        raise ValueError(f"Action doit avoir 3 ou 4 dimensions, reçu {action.ndim}")

In [3]:
#Récupération des données

folder_data = "../data/"

states_files = [
    'training-data-heuristic-vs-laheuristic2-states.pkl',
    'training-data-heuristic-vs-laheuristic3-2-states.pkl',
    'training-data-heuristic-vs-laheuristic3-states.pkl',
    'training-data-laheuristic2-vs-laheuristic3-states.pkl',
    'training-data-random-vs-laheuristic3-states.pkl'
]

policy_files = [
    'training-data-heuristic-vs-laheuristic2-actions.pkl',
    'training-data-heuristic-vs-laheuristic3-2-actions.pkl',
    'training-data-heuristic-vs-laheuristic3-actions.pkl',
    'training-data-laheuristic2-vs-laheuristic3-actions.pkl',
    'training-data-random-vs-laheuristic3-actions.pkl'
]

states = []
policies = []

for i in range(len(states_files)):
    filename_states = folder_data+states_files[i]
    filename_policies = folder_data+policy_files[i]
    states += RegularDataTrainer.getTrainedData(filepath=filename_states)
    policies += RegularDataTrainer.getTrainedData(filepath=filename_policies)
    print(f"File {filename_states} loaded !")
    print(f"File {filename_policies} loaded !")

print("\n\nTotal :")
print(f"{len(states)} states")
print(f"{len(policies)} policies")

states = np.array(states)
print(states.shape)

#On est en (10, 5, 5) le réseau attend du (5, 5, 10) il faut transposer
states = np.transpose(states, (0, 2, 3, 1))
print(states.shape) #Maintenant on est bien (en 5,5,10)

File ../data/training-data-heuristic-vs-laheuristic2-states.pkl loaded !
File ../data/training-data-heuristic-vs-laheuristic2-actions.pkl loaded !
File ../data/training-data-heuristic-vs-laheuristic3-2-states.pkl loaded !
File ../data/training-data-heuristic-vs-laheuristic3-2-actions.pkl loaded !
File ../data/training-data-heuristic-vs-laheuristic3-states.pkl loaded !
File ../data/training-data-heuristic-vs-laheuristic3-actions.pkl loaded !
File ../data/training-data-laheuristic2-vs-laheuristic3-states.pkl loaded !
File ../data/training-data-laheuristic2-vs-laheuristic3-actions.pkl loaded !
File ../data/training-data-random-vs-laheuristic3-states.pkl loaded !
File ../data/training-data-random-vs-laheuristic3-actions.pkl loaded !


Total :
226287 states
226287 policies
(226287, 10, 5, 5)
(226287, 5, 5, 10)


In [4]:
#On applique one Hot Encoder sur les actions (policies)
policies = action_to_one_hot(policies)

#Aplatir pour correspondre à la sortie du modèle
policies = policies.reshape(-1, 5 * 5 * 52)  # (batch, 1300)

print(policies.shape)

(226287, 1300)


In [ ]:
network = CNNPlayer()
#network.compile_for_supervised_policy(learning_rate=0.001)
network.load_weights("../saved-models/CNNPlayer-withdropout-weights.weights.h5")

Gelé 7 layers de la tête de valeur
Modèle compilé pour entraînement supervisé (policy seulement, label_smoothing=0.1)


/home/lgerard/python/onitama-rl/.venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 102 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [7]:
x_train = np.array(states)
y_policy = policies
y_value = np.zeros((len(x_train), 1))

In [8]:
def predict_action(network, state, deterministic=True, valid_actions_mask=None):
    """
    Prédit une action à partir d'un état
    
    Args:
        network: Le réseau de neurones
        state: (5, 5, 10) - un état (sans batch dimension)
        deterministic: Si True, prend l'action avec le score max
                      Si False, échantillonne selon les probabilités
        valid_actions_mask: (5, 5, 52) optionnel - masque des actions valides
        
    Returns:
        action: [col, ligne, move_id]
        probability: probabilité de cette action
    """
    # 1. Ajouter la dimension batch si nécessaire
    if len(state.shape) == 3:
        state_batch = np.expand_dims(state, axis=0)  # (1, 5, 5, 10)
    else:
        state_batch = state
    
    # 2. Prédiction
    policy_logits, value = network.predict(state_batch, training=False)
    
    # 3. Retirer la dimension batch
    policy_logits = policy_logits.numpy()
    policy_logits = policy_logits[0]  # (5, 5, 52)
    value = value[0, 0]  # Scalar
    
    # 4. Masquer les actions invalides (optionnel mais IMPORTANT)
    if valid_actions_mask is not None:
        large_negative = -1e9
        masked_logits = np.where(
            valid_actions_mask,
            policy_logits,
            large_negative
        )
    else:
        masked_logits = policy_logits
    
    # 5. Aplatir en (1300,) pour faciliter le traitement
    flat_logits = masked_logits.reshape(-1)  # (5*5*52 = 1300,)
    
    # 6. Convertir en probabilités avec softmax
    flat_probs = softmax(flat_logits)
    
    # 7. Sélectionner l'action
    if deterministic:
        # Prendre l'action avec la probabilité max
        best_index = np.argmax(flat_probs)
    else:
        # Échantillonner selon les probabilités
        best_index = np.random.choice(len(flat_probs), p=flat_probs)
    
    # 8. Décoder l'index en [col, ligne, move_id]
    col = best_index // (5 * 52)
    ligne = (best_index // 52) % 5
    move_id = best_index % 52
    
    action = [int(col), int(ligne), int(move_id)]
    probability = flat_probs[best_index]
    
    return action, probability, value


def softmax(x):
    """Calcule le softmax de manière stable numériquement"""
    exp_x = np.exp(x - np.max(x))  # Soustraire le max pour stabilité
    return exp_x / exp_x.sum()

In [9]:
#Vérification des prédictions pour les premiers états

for i in range(10):
    # Prédire l'action (déterministe)
    state = x_train[i]
    action_reelle = y_policy[i]
    action, prob, value = predict_action(network, state, deterministic=True)

    print("---------------------")
    print(f"État: {state.shape}")
    print(f"Action prédite: {action}")
    print(f"  → Colonne: {action[0]}")
    print(f"  → Ligne: {action[1]}")
    print(f"  → Mouvement: {action[2]}")
    print(f"Probabilité: {prob:.4f}")

    print(decode_flat_policy(action_reelle))


---------------------
État: (5, 5, 10)
Action prédite: [0, 4, 42]
  → Colonne: 0
  → Ligne: 4
  → Mouvement: 42
Probabilité: 0.6359
[0, 4, 42]
---------------------
État: (5, 5, 10)
Action prédite: [1, 3, 46]
  → Colonne: 1
  → Ligne: 3
  → Mouvement: 46
Probabilité: 0.1833
[4, 4, 30]
---------------------
État: (5, 5, 10)
Action prédite: [1, 4, 46]
  → Colonne: 1
  → Ligne: 4
  → Mouvement: 46
Probabilité: 0.9439
[1, 4, 46]
---------------------
État: (5, 5, 10)
Action prédite: [0, 4, 50]
  → Colonne: 0
  → Ligne: 4
  → Mouvement: 50
Probabilité: 0.2670
[2, 4, 50]
---------------------
État: (5, 5, 10)
Action prédite: [3, 3, 29]
  → Colonne: 3
  → Ligne: 3
  → Mouvement: 29
Probabilité: 0.5322
[3, 3, 29]
---------------------
État: (5, 5, 10)
Action prédite: [0, 4, 19]
  → Colonne: 0
  → Ligne: 4
  → Mouvement: 19
Probabilité: 0.4079
[0, 4, 19]
---------------------
État: (5, 5, 10)
Action prédite: [1, 3, 49]
  → Colonne: 1
  → Ligne: 3
  → Mouvement: 49
Probabilité: 0.4759
[1, 3, 49]